Librerías

In [1]:
import pandas as pd
import json
import chardet
import ast
from pandas import json_normalize
import pickle

De formaton JSON a CSV

In [2]:
#Se busca el ecoding del archivo 'australian_user_reviews.json'
with open(r'C:\Users\Nahir\Desktop\JSON\australian_users_items.json', 'rb') as file:
    rawdata = file.read()
    result = chardet.detect(rawdata)
print(result['encoding'])

MacRoman


In [ ]:
#Se leen los datos del archivo 'australian_user_reviews.json'
rows = []
with open(r'C:\Users\Nahir\Desktop\JSON\australian_users_items.json', encoding='MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))
# Se convierte el contenido a un dataframe
df_items = pd.DataFrame(rows)
# Se guarda en formato CSV
df_items.to_csv('australian_users_items.csv', index=False)

Explorando los datos

In [3]:
#Se define el df items.
df_i=pd.read_csv(r'C:\Users\Nahir\Desktop\CSV\australian_users_items.csv', low_memory=False)
df_i.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [13]:
#Se explora el df de manera general.
df_i.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  int64 
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.4+ MB


In [14]:
#Se calcula la cantidad de duplicados.
df_i.duplicated().sum()

657

In [15]:
#Se eleiminan los duplicados.
df_i.drop_duplicates(inplace=True)

In [16]:
#Se explora el contenido de la columna 'items'.
df_i['items']

0        [{'item_id': '10', 'item_name': 'Counter-Strik...
1        [{'item_id': '10', 'item_name': 'Counter-Strik...
2        [{'item_id': '1200', 'item_name': 'Red Orchest...
3        [{'item_id': '10', 'item_name': 'Counter-Strik...
4        [{'item_id': '300', 'item_name': 'Day of Defea...
                               ...                        
88305    [{'item_id': '413850', 'item_name': 'CS:GO Pla...
88306    [{'item_id': '11020', 'item_name': 'TrackMania...
88307                                                   []
88308    [{'item_id': '304930', 'item_name': 'Unturned'...
88309                                                   []
Name: items, Length: 87653, dtype: object

In [17]:
#Se verifica el tipo de dato de la columna 'items' para saber como proceder.
unique_i = df_i['items'].apply(type).unique()
print(unique_i)

[<class 'str'>]


Debido a que la columna items contiene datos anidados, desplegamos esos datos en diferentes columnas nuevas y las agregamos al df existente.

In [18]:
# Convertir los strings a diccionarios
df_i['items'] = df_i['items'].apply(ast.literal_eval)
# Extraer la información de los diccionarios en la columna 'items'
df_i_i = json_normalize(df_i['items'].apply(lambda x: x[0] if x else {}))
# Concatenar las columnas desglosadas con el DataFrame original
df_i2 = pd.concat([df_i[['user_id', 'user_url','items_count','steam_id']], df_i_i], axis=1)

# Verificar el nuevo DataFrame con las columnas desglosadas y las columnas originales
df_i2.head()

,user_id,user_url,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277.0,7.656120e+16,10,Counter-Strike,6.0,0.0
1,js41637,http://steamcommunity.com/id/js41637,888.0,7.656120e+16,10,Counter-Strike,0.0,0.0
2,evcentric,http://steamcommunity.com/id/evcentric,137.0,7.656120e+16,1200,Red Orchestra: Ostfront 41-45,923.0,0.0
3,Riot-Punch,http://steamcommunity.com/id/Riot-Punch,328.0,7.656120e+16,10,Counter-Strike,0.0,0.0
4,doctr,http://steamcommunity.com/id/doctr,541.0,7.656120e+16,300,Day of Defeat: Source,1131.0,0.0


In [19]:
#Se explora la composición del df con las nuevas columnas.
df_i2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88310 entries, 0 to 77045
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           87653 non-null  object 
 1   user_url          87653 non-null  object 
 2   items_count       87653 non-null  float64
 3   steam_id          87653 non-null  float64
 4   item_id           70939 non-null  object 
 5   item_name         70939 non-null  object 
 6   playtime_forever  70939 non-null  float64
 7   playtime_2weeks   70939 non-null  float64
dtypes: float64(4), object(4)
memory usage: 6.1+ MB


Considerando los endpoints a desarrolar, se consideran irrelevantes las columnas 'items_count', 'user_url','playtime_2weeks', por lo que se eliminan. Tambien de rellenan los nulos de la columna 'playtime'forever' para no generar posibles problemas en las funciones.

In [20]:
df_i2=df_i2.drop(['items_count', 'user_url','playtime_2weeks'],axis=1)
df_i2['playtime_forever'].fillna(0, inplace=True)
df_i2.head()

,user_id,steam_id,item_id,item_name,playtime_forever
0,76561197970982479,7.656120e+16,10,Counter-Strike,6.0
1,js41637,7.656120e+16,10,Counter-Strike,0.0
2,evcentric,7.656120e+16,1200,Red Orchestra: Ostfront 41-45,923.0
3,Riot-Punch,7.656120e+16,10,Counter-Strike,0.0
4,doctr,7.656120e+16,300,Day of Defeat: Source,1131.0


In [21]:
#Se explora la composición del df luego de haber eliminado las columnas anteriores.
df_i2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88310 entries, 0 to 77045
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           87653 non-null  object 
 1   steam_id          87653 non-null  float64
 2   item_id           70939 non-null  object 
 3   item_name         70939 non-null  object 
 4   playtime_forever  88310 non-null  float64
dtypes: float64(2), object(3)
memory usage: 4.0+ MB


In [22]:
#Se eliminan los nulos de las columnas que quedaron.
df_i2.dropna(inplace=True)
df_i2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70378 entries, 0 to 87651
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           70378 non-null  object 
 1   steam_id          70378 non-null  float64
 2   item_id           70378 non-null  object 
 3   item_name         70378 non-null  object 
 4   playtime_forever  70378 non-null  float64
dtypes: float64(2), object(3)
memory usage: 3.2+ MB


### PKL

In [23]:
#Se guarda el df en un archivo con formato PKL.
pkl_file = open('items.pkl', 'ab')
pickle.dump(df_i2, pkl_file)
pkl_file.close()